# Using the ORM to store several boreholes

Imports

In [1]:
from os import remove
from utils.io import boreholes_from_files, striplog_from_text
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [2]:
remove('tmp_files/tfe_orm_db.db')

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2021-04-30 21:29:25,443 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-30 21:29:25,446 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 21:29:25,448 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-30 21:29:25,450 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 21:29:25,453 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-04-30 21:29:25,454 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 21:29:25,457 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-04-30 21:29:25,459 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 21:29:25,461 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-04-30 21:29:25,462 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 21:29:25,464 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-04-30 21:29:25,465 INFO sqlalchemy.engine.base.Engine ()
2021-04-30

### Create the objects from las files

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F1':ROOT_DIR+'/data/test.las'}#, 'F2':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [11]:
boreholes, components, intv_comp = boreholes_from_files(borehole_dict, verbose=0)
len(boreholes)



File 0 processing...
bh: F2, pos: 1
bh: F3, pos: 2
bh: F4, pos: 3


4

Create the session

In [12]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [13]:
p = Project(session)
p.add_components(components)

2021-04-30 21:30:57,738 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-30 21:30:57,740 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-04-30 21:30:57,742 INFO sqlalchemy.engine.base.Engine ()
2021-04-30 21:30:57,750 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-04-30 21:30:57,751 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2021-04-30 21:30:57,755 INFO sqlalchemy.engine.base.Engine COMMIT
2021-04-30 21:30:57,761 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-30 21:30:57,762 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021

Add boreholes into the database

In [14]:
for bh in boreholes:
    p.add_borehole(bh)

2021-04-30 21:31:01,402 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-04-30 21:31:01,405 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.47647058823532, 131.47647058823532, 0.0, 0.0), (49, 135.9, 135.9, 135.9, 0.0, 0.0))
2021-04-30 21:31:01,410 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, leng

2021-04-30 21:31:01,505 INFO sqlalchemy.engine.base.Engine (14,)
2021-04-30 21:31:01,508 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,509 INFO sqlalchemy.engine.base.Engine (15,)
2021-04-30 21:31:01,511 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,512 INFO sqlalchemy.engine.base.Engine (16,)
2021-04-30 21:31:01,514 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-04-30 21:31:01,570 INFO sqlalchemy.engine.base.Engine (35,)
2021-04-30 21:31:01,571 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,572 INFO sqlalchemy.engine.base.Engine (36,)
2021-04-30 21:31:01,574 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,575 INFO sqlalchemy.engine.base.Engine (37,)
2021-04-30 21:31:01,577 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-04-30 21:31:01,717 INFO sqlalchemy.engine.base.Engine ('F2',)
2021-04-30 21:31:01,719 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,720 INFO sqlalchemy.engine.base.Engine (50,)
2021-04-30 21:31:01,722 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,723 INFO sqlalchemy.engine.base.Engine (51,)
2021-04-30 21:31:01,726 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Posi

2021-04-30 21:31:01,780 INFO sqlalchemy.engine.base.Engine (70,)
2021-04-30 21:31:01,783 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,784 INFO sqlalchemy.engine.base.Engine (71,)
2021-04-30 21:31:01,786 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:01,788 INFO sqlalchemy.engine.base.Engine (72,)
2021-04-30 21:31:01,790 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

...

2021-04-30 21:31:02,004 INFO sqlalchemy.engine.base.Engine (105,)
2021-04-30 21:31:02,007 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,007 INFO sqlalchemy.engine.base.Engine (106,)
2021-04-30 21:31:02,010 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,011 INFO sqlalchemy.engine.base.Engine (107,)
2021-04-30 21:31:02,013 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

2021-04-30 21:31:02,068 INFO sqlalchemy.engine.base.Engine (126,)
2021-04-30 21:31:02,070 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,071 INFO sqlalchemy.engine.base.Engine (127,)
2021-04-30 21:31:02,073 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,074 INFO sqlalchemy.engine.base.Engine (128,)
2021-04-30 21:31:02,076 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

2021-04-30 21:31:02,130 INFO sqlalchemy.engine.base.Engine (147,)
2021-04-30 21:31:02,132 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,133 INFO sqlalchemy.engine.base.Engine (148,)
2021-04-30 21:31:02,135 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,135 INFO sqlalchemy.engine.base.Engine (149,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
2021-04-30 21:31:02,203 INFO sqlalc

2021-04-30 21:31:02,275 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,276 INFO sqlalchemy.engine.base.Engine (161,)
2021-04-30 21:31:02,278 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,279 INFO sqlalchemy.engine.base.Engine (162,)
2021-04-30 21:31:02,281 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positi

2021-04-30 21:31:02,336 INFO sqlalchemy.engine.base.Engine (181,)
2021-04-30 21:31:02,339 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,339 INFO sqlalchemy.engine.base.Engine (182,)
2021-04-30 21:31:02,341 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-04-30 21:31:02,342 INFO sqlalchemy.engine.base.Engine (183,)
2021-04-30 21:31:02,344 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

In [15]:
p.commit()

2021-04-30 21:31:03,790 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-04-30 21:31:03,792 INFO sqlalchemy.engine.base.Engine (135.9, 'F4')
2021-04-30 21:31:03,796 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
p.boreholes[0].intervals[0].description

2021-04-30 21:31:04,301 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-30 21:31:04,305 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-04-30 21:31:04,307 INFO sqlalchemy.engine.base.Engine ('F1',)
2021-04-30 21:31:04,310 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-04-30 21:31:04,311 INFO sqlalchemy.engine.base.Engine ('F1',)


'Anhydrite'

In [17]:
session.close()

2021-04-30 21:31:08,051 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [18]:
p.plot3d(x3d=True)